自分の学習用に記録を作成。

今回、初めて自然言語処理に挑戦するため、データセットのEDAおよびrobertの実装を行う。

ERIK BRUINさんのNLP on Student Writing: EDAを参考にEDAを行なっていく。
https://www.kaggle.com/erikbruin/nlp-on-student-writing-eda

In [ ]:
#モジュールのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

In [ ]:
#データの読み込み
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train_txt = glob('../input/feedback-prize-2021/train/*.txt') 
test_txt = glob('../input/feedback-prize-2021/test/*.txt')
sample_submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

train.csvデータ144293行8列
トレーニングセット内のエッセイの注釈付きバージョンを含む.csvファイル
それぞれのデータの意味は下記の通りになる

id:エッセイ応答番号

discourse_id:談話要素のIDコード

discourse_start:エッセイの応答で談話要素が始まる文字の位置

discourse_end:談話要素がエッセイ応答で終了する文字の位置

discourse_text:談話要素のテキスト

discourse_type:談話要素の分類

discourse_type_num:談話要素の列挙されたクラスラベル

predictionstring:予測に必要なトレーニングサンプルの単語インデックス

In [ ]:
#traindata
print(train.shape)
train.head()

discourse_type_numは談話要素の列挙されたクラスラベルは４２種類

discourse_typeは７種類ある

In [ ]:
#predivtionstringのユニーク数
print(train["predictionstring"].nunique())
print("------------------------------------")
print(train["discourse_type_num"].nunique())
print(train["discourse_type_num"].unique())
print("------------------------------------")
print(train["discourse_type"].unique())

In [ ]:
print(train["discourse_text"][0])
print(train["discourse_type_num"][0])
print(train["predictionstring"][0])
print("--------------------------------")
print(train["discourse_text"][1])
print(train["discourse_type_num"][1])
print(train["predictionstring"][1])

In [ ]:
#試しにID0000D23A521Aのデータを抽出
train.query('id =="0000D23A521A"')

In [ ]:
!cat ../input/feedback-prize-2021/train/0000D23A521A.txt

predictionstringの数とdiscourse_textの単語数が正しいか確認

In [ ]:
#テキストをスペースで分割し数を計算
train["discourse_len"] = train["discourse_text"].apply(lambda x:len(x.split()))
#predictionstringをスペースで分割し数を計算
train["predictionstring_len"] = train["predictionstring"].apply(lambda x:len(x.split()))

train["len_predict"] = 0

#discourse_lenとpredictionstring_lenで差があるか確認
for i in range(0,len(train["discourse_len"])):
    if train["discourse_len"][i] == train["predictionstring_len"][i]:
        train["len_predict"][i] = 0
    else:
        train["len_predict"][i] = 1

#異なっている件数を確認
train["len_predict"].sum()

In [ ]:
train.head()

In [ ]:
#len_predictが1のものを抽出
df = train.query("len_predict == 1")

#id4E1C636ABEADを確認
df.query("id == '4E1C636ABEAD'")

In [ ]:
!cat ../input/feedback-prize-2021/train/4E1C636ABEAD.txt

単語の数を数えてみると正しいのはdiscourse_lenであり、predictionstringの中には合計468件の誤りがある

type別の誤り発生率は誤差0.005であり優位な差はないと考える

In [ ]:
#len_predictはtype毎に差があるか確認
df1 = df.groupby("discourse_type").agg({"len_predict":"sum"}).reset_index()
#trainデータで総数をカウント
df2 = train.groupby("discourse_type").agg({"id":"count"}).reset_index()
#df1とdf2を結合
df3 = pd.merge(df1,df2,on="discourse_type",how="left")
#typeごとの割合を計算
df3["len_predict/id"] = df3["len_predict"] / df3["id"]
print(df3)

discourse_lenの数はtypeによって優位な差があるのか確認

Evidence、Concluding Statement、Leadはその他のtypeと比べて単語数が多い

In [ ]:
#discourse_lenの平均をtype毎に計算
df4 = train.groupby("discourse_type").agg({"discourse_len":"mean"}).reset_index()
print(df4)
plt.bar(df4["discourse_type"],df4["discourse_len"])

In [ ]:
#欠損値の確認
train.isnull().sum()

In [ ]:
#discourse_end,discourse_startの平均をtypeで計算
df5 = train.groupby("discourse_type")[['discourse_end', 'discourse_start']].mean().reset_index().sort_values(by = 'discourse_start', ascending = False)
df5.plot(x='discourse_type',
        kind='barh',
        stacked=False,
        title='Average start and end position absolute',
        figsize=(12,4))
plt.show()

In [ ]:
from tqdm import tqdm
#testデータの読み込み
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
# test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts.head()

In [ ]:
#trainデータのh読み込み
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
# train_texts['text'] = test_texts['text'].apply(lambda x:x.split())
train_text_df.head()

In [ ]:
train_df = train.copy()
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split(' ').__len__()
    start = -1
    entities = []
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split(' ')
#         print(j[1]['predictionstring'],'###' ,len(list_ix))
        ent = [f"I-{discourse}" for ix in list_ix]
        ent[0] = f"B-{discourse}"
        ds = int(list_ix[0])
        de = int(list_ix[-1])
        if start < ds-1:
            ent_add = ['O' for ix in range(int(ds-1-start))]
            ent = ent_add + ent
#         print(len(entities))
#         print(ent, len(ent))
        entities.extend(ent)
#         print(len(entities))
        start = de
    if len(entities) < total:
        ent_add = ["O" for ix in range(total-len(entities))]
        entities += ent_add
    else:
        entities = entities[:total]
#     print(i[1]['id'],'@@@@@@@@' ,i[1]['text'].split(' ').__len__(), len(entities))
    all_entities.append(entities)
#     if len(all_entities) > 100:
#         break

In [ ]:
train_text_df['entities'] = all_entities
train_text_df.head()

**そもそも、BERTとは**

2018年にGoogleから発表された自然言語処理モデルであり、多様な自然言語処理において当時の最高スコアを叩き出していた。
特徴は「文脈を読むことが可能」であり、文章を文頭と文末の双方向から学習することで文脈を読むことを実現した。

BERTの仕組み
入力されたシーケンスから別のシーケンスを予測します。
事前学習モデルであり、入力されたラベルが付与されていない、分散表現をTronsformerが処理することによって学習をします。

・Masked language model
入力文の15%の単語を確率的に別の単語で置き換え、文脈から置き換える前の単語を予測させます。具体的には、選択された15%のうち、80%は[MASK]に置き換えるマスク変換、10%をランダムな別の単語に変換、残りの10%はそのままの単語にします。置換された単語を周りの文脈から当てるタスクを解くことで、単語に対応する文脈情報を学習する。

・Next sentence prediction
2つの入力文に対して「その2文が隣り合っているか」を当てるよう学習します。これにより、2つの文の関係性を学習できます。文の片方を50%の確率で他の文に置き換え、それらが隣り合っているか（isNext）隣り合っていない（notNext）か判別することによって学習します。（2文を[SEP]というトークンで分け、isNextかnotNextか分類するために[CLS]というトークンが用意されます。）

**RoBERTについて**

RoBERTのアイディアは"BERTの持っている力を発揮するために、もっと上手く、たくさん学習すればもっと賢くなる"、という考えの元生み出されました。

・事前学習の設定
BERTの実装では最初の時点でマスクをするがRoBERTはdynamic maskingとして学習する度にマスクをするようにしている。

・Nect sentence predicition
FULL-SENTENCES
一つ以上のドキュメントから連続した512単語をインプットとします。
1つのドキュメントが512単語以下の場合は、違うドキュメントの最初からサンプリングします。
DOC-SENTENCES
FULL-SENTENCESと似ていますが、ドキュメントをまたぎません。
通常、512単語より短いので、FULL-SENTENCESと同水準の単語数になるようにバッチサイズをダイナミックに増やします。
基本的にはDOC-SENTENCESを使用する。

・バッチサイズ
BERTでは、バッチサイズ256だが、非常に大きいバッチサイズで学習すると精度が向上するという結果がある。

・Text Encoding
BERTではByte-Pair Encodingという手法を使って単語を作成している。
BPEは単語に分割し、出現頻度が高いものは単語のままで、出現頻度が低いものは文字単位に置き換えることにより、未知語をなくそうというものです。
文字単位からバイト単位に変えたが結果は若干悪化したとのこと。



**RoBertaを実装。**

自力で実装する知識がなため下記のノードブックを写経。

https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533/notebook

In [ ]:
#必要なライブラリの読み込み
import random
import glob
#進捗状態をプログレスバーで表示
from tqdm import tqdm

import torch
from torch.utils.data import Dataset,DataLoader
from transformers import RobertaTokenizerFast, RobertaForTokenClassification
#ファインチューニングと性能評価を効率的に行うためライブラリ
import pytorch_lightning as pl

import pdb
from torch import cuda
from sklearn.metrics import accuracy_score

#Robertaの事前学習モデル
MODEL_NAME = "../input/roberta-base"

In [ ]:

config = {'model_name': '/kaggle/input/roberta-base/',
         'max_length': 512,
         'train_batch_size':8,
         'valid_batch_size':16,
         'epochs':3,
         'learning_rate':1e-05,
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

In [ ]:
#ラベルリストの作成
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [ ]:
print(labels_to_ids)
print(ids_to_labels)

In [ ]:
#トークナイザのロード
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_NAME)
#モデル作成
model = RobertaForTokenClassification.from_pretrained(config['model_name'], num_labels=len(output_labels))

In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index]
        word_labels = self.data.entities[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_pretokenized=True, 
#                                   is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
#         pdb.set_trace()
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
#         print(len(sentence), len(labels))
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
#             print(idx)
            if mapping[0] != 0 and mapping[0] != encoding['offset_mapping'][idx-1][1]:
            # overwrite label
#             pdb.set_trace()
#             print(mapping)
#             print(encoded_labels.shape, len(labels), idx, i)
                try:
                    encoded_labels[idx] = labels[i]
                except:
                    pass
                i += 1
            else:
                if idx==1:
    #                 print(idx)
                    encoded_labels[idx] = labels[i]
                    i += 1
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

train_text_dfを8対2で検証データと分割

dataset classでencodingを実施しdataloaderに入れれる形にする。

In [ ]:
data = train_text_df[['text', 'entities']]
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, config['max_length'])
testing_set = dataset(test_dataset, tokenizer, config['max_length'])

In [ ]:
#datasetの中身を確認
print(training_set[0])

encodingしたdataを使用しdataloaderを作成。※dataloaderはデータセットからミニバッチを取り出すためのもの。

In [ ]:
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

test_params = {'batch_size': config['valid_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
device = config['device']

In [ ]:
model.to(device)
inputs = training_set[2]
#unsqueeze(dim)：元のテンソルを書き換えずに、次元を増やしたテンソルを返す
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

#GPU/CPUの切り替え
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels,
               return_dict=False)
initial_loss = outputs[0]
initial_loss

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'])

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(config['epochs']):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                                     return_dict=False)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
sentence = "@HuggingFace is a company based in New York, but is also has employees working in Paris"
model.eval()
def inference(sentence):
    inputs = tokenizer(sentence,
#                         is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=config['max_length'],
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask, return_dict=False)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] != 0 and mapping[0] != off_list[idx-1][1]:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [ ]:
# test_texts = train_text_df['text'].tolist()[:10]
y_pred = []

for i, t in enumerate(test_texts['text'].tolist()):
    o,o_t = inference(t)
    y_pred.append(o)
    l = train_text_df['entities'][i]

In [ ]:
final_preds = []
import pdb
for i in tqdm(range(len(test_texts))):
#     pdb.set_trace()
    idx = test_texts.id.values[i]
#     pred = ['']*len(test_texts[i])

#     for j in range(len(y_pred[i])):
#         if words[i][j] != None:
#             pred[words[i][j]] = labels[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in y_pred[i]]
#     print(pred)
    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
#         pdb.set_trace()
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'O' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
print(final_preds[1])

In [ ]:
print(len(final_preds))
test_df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test_df

In [ ]:
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)